# Regular wall od edge dislocations

## Define geometry

Choose an imaged reflection, surface normal and orientation of the dislocation wall

In [2]:
### Do geometry calculation
import numpy as np
lmbd = 12.398 / 17.00 * 1e-7
latt_param = 4.0478e-7 # Cubic!
hkl = np.array([-1 ,1 ,-1])

#################### CALCULATE CRYSTAL BASIS #######################
# Crystal reference frame vectors in the crystal lattice basis
z = hkl / np.linalg.norm(hkl) # this is the scattering vector
x = np.array([1, 0, -1])/np.sqrt(2) # This is the surface of the crystal. ###!!! YOU AHVE TO PICK THIS NORMAL TO Q AT THE MOMENT!!!###
# y = np.array([-1, -1, 1])/np.sqrt(3) # This one comes for free
y = np.cross(x, z)


# Dislocation coordinate system in the crystal lattice basis
b = np.array([1, -1, 0])/np.sqrt(2)
n = np.array([-1, -1, 1])/np.sqrt(3)
t = np.array([1, 1, 2])/np.sqrt(6)
disloc_to_latt = np.array([b, n, t]).transpose() # <---------------------THIS ONE ACTUALLY GETS USED IN THE SIMULATION
latt_to_disloc = disloc_to_latt.T

# Calculate rotation matrices between lattice sysstem and reference system
ref_to_latt  = np.array([x, y, z]).transpose()
latt_to_ref = ref_to_latt.T


# Lattice matrix in lattice system
A = np.eye(3)*latt_param
B = 2*np.pi*np.linalg.inv(A).T
Q = np.dot(B, hkl)
theta = np.arcsin(np.linalg.norm(Q)/4/np.pi*lmbd)      # internally we keep angles in rad, 

# Reference system to laboratory system
ref_to_lab = np.array([[np.cos(theta), 0, -np.sin(theta)],[0, 1, 0],[np.sin(theta), 0, np.cos(theta)]])

# Final U matrix (lattice to laboratory)
U = ref_to_lab @ latt_to_ref
AU = latt_param * U
BU = 2*np.pi*np.linalg.inv(AU).T

# Other intemediary rotationmatrices, only calculated so I can print and compare with Sina's results
lab_to_latt = U.T
lab_to_disloc = latt_to_disloc @ lab_to_latt  
disloc_to_lab = lab_to_disloc.T
print(disloc_to_lab)

###### Calculate Scattering angle and wavevectors in crystal reference frame ########
# Simulations use different ordering of the x-y-z than the geometry calculation here.  (I chose z as the optical axis originally)
sim_to_ref = np.array([[0,0,1],[0,-1,0],[1,0,0]] )
ref_to_sim = sim_to_ref.T
U_sim = ref_to_sim @ latt_to_ref

R_disloc_sim = U_sim @ disloc_to_latt
UA_sim = U_sim @ A


print(f'2θ = {2*theta*180/np.pi:.3f}°')  # but always print in degrees, please
k_0_ref = np.array([-np.sin(theta), 0, np.cos(theta) ])*2*np.pi/lmbd
print(f'k0 = ({k_0_ref[0]*1e-6:.2f}, {k_0_ref[1]*1e-6:.2f}, {k_0_ref[2]*1e-6:.2f}) 1/nm')
Q = 2*np.pi*np.linalg.inv(UA_sim).T @ hkl
print(f'Q = ({Q[0]*1e-6:.2f}, {Q[1]*1e-6:.2f}, {Q[2]*1e-6:.2f}) 1/nm')


print(y)

[[ 0.62127576 -0.75448536 -0.21158514]
 [-0.28867513 -0.47140452  0.83333333]
 [-0.72847999 -0.45665043 -0.51067336]]
2θ = 17.953°
k0 = (-13.44, 0.00, 85.10) 1/nm
Q = (26.89, 0.00, 0.00) 1/nm
[0.40824829 0.81649658 0.40824829]


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


fig = plt.figure(figsize = (4,5))
ax = Axes3D(fig)


def init():
    

    # dislocation frame
    s = 0.3 # scale of the arrows
    
    ax.quiver( 0.5, 0.5, 0.5, lab_to_disloc[0,2]*s, lab_to_disloc[0,1]*s, lab_to_disloc[0,0]*s, color='r')
    ax.quiver( 0.5, 0.5, 0.5, lab_to_disloc[1,2]*s, lab_to_disloc[1,1]*s, lab_to_disloc[1,0]*s, color='g')
    ax.quiver( 0.5, 0.5, 0.5, lab_to_disloc[2,2]*s, lab_to_disloc[2,1]*s, lab_to_disloc[2,0]*s, color='b')
    plt.legend(['b', 'n', 't'])

    ax.quiver( 0., 0., 1.2, ref_to_lab[0,2]*s, ref_to_lab[0,1]*s, ref_to_lab[0,0]*s, color='k')
    ax.quiver( 0., 0., 1.2, ref_to_lab[1,2]*s, ref_to_lab[1,1]*s, ref_to_lab[1,0]*s, color='k')
    ax.quiver( 0., 0., 1.2, ref_to_lab[2,2]*s, ref_to_lab[2,1]*s, ref_to_lab[2,0]*s, color='k')


    # Crystal surfaces
    xx = np.linspace(0, 1, 2)
    yy = np.linspace(0, 1, 2)
    xx, yy = np.meshgrid(xx,yy, indexing ='ij')

    ax.plot_surface(0.3*np.ones(xx.shape), yy, xx, alpha=0.2, color = [0.5, 0.5, 0.5])
    ax.plot_surface(0.7*np.ones(xx.shape), yy, xx, alpha=0.2, color = [0.5, 0.5, 0.5])


    zz = np.linspace(0, 1, 2)
    yy = np.linspace(0.4, 0.6, 2)
    zz, yy = np.meshgrid(xx,yy, indexing ='ij')
    ax.plot_surface(zz, yy, -np.tan(theta)*zz+0.6 , alpha=0.5)


    # Dislocation lines
    for number in range(-3, 3):
        
        s = 0.4
        pos = np.array((0.5, 0.5, 0.5)) + number*0.03*lab_to_disloc[1,:] - s/2*lab_to_disloc[2,:]
        
        #ax.quiver( *np.flip(pos), lab_to_disloc[2,2], lab_to_disloc[2,1], lab_to_disloc[2,0], color='k')
        
        start = np.flip(pos); end = np.flip(pos + s*lab_to_disloc[2,:])
        
        plt.plot(*( (start[ii], end[ii]) for ii in range(3) ) ,color = 'k')



    ax.set_xlabel('$x_c$')
    ax.set_ylabel('$y_c$')
    ax.set_zlabel('$z_c$')

    ax.set_xlim3d((0,1))
    ax.set_ylim3d((-0.2,1.2)) # This is a BAD hack to make the axes appear orthonormal. 3D plotting in pyplot is  an unmitigated disaster ATM, because they decided to un-implement axis='equal', which did some thing wrong, but was WAAAAAAY better that what they have now instead.
    ax.set_zlim3d((0,1))
 
    return [fig]
    
# def animate(i):
#     ax.view_init(elev=10., azim=i)
#     return [fig]
    

# # Animate
# anim = FuncAnimation(fig, animate, init_func=init, frames=360, interval=20, blit=True)
# # Save
# anim.save('basic_animation.mp4', fps=30, extra_args=['-vcodec', 'libx264'])



init()

def animate(i):
    ax.view_init(elev=25., azim=i)
    return [fig]
animate(300)

plt.savefig('geom_view.svg')

plt.show()


## Computational parameters and energy bandwidth


In [ ]:
### Write values to a parameter file
import sys
sys.path.insert(1, '/Documents/PhD DTU/Geometrical Optics/Wave propagation/forward_simulation_polychromatic-master/')
import dfxm_fwrd_sim.parameter_parser as par
root_dir = 'forward_simulation_polychromatic-master/al_1.0_mu/'
par_fn = 'try_al_3.ini'
params = par.init_parameter_dictionary(root_dir + par_fn)
params['I/O']['root_dir'] = root_dir

# Parameter parser only takes vectors
params['Material']['a'] = UA_sim[:,0]
params['Material']['b'] = UA_sim[:,1]
params['Material']['c'] = UA_sim[:,2]

######## Geometry of integration ######
params['Geometry']['grid_shape'] = [1024, 512, 1001]
params['Geometry']['step_sizes'] = [40e-6, 40e-6, 100e-6]
params['Geometry']['hkl'] = hkl
params['Geometry']['Q'] = Q

# Make energy axis
sig_E_rel = 1e-4 
n_E = 11
relative_energy_list = np.linspace(-3*sig_E_rel, 3*sig_E_rel, n_E)
rel_weight = np.exp(-relative_energy_list**2/2/sig_E_rel**2)
abs_weight = rel_weight / np.sum(rel_weight)


params['Beam'] = {'lmbd_ref':lmbd, 'k_0_ref':k_0_ref, 'N_modes':len(relative_energy_list)}


# Loop over modes and create a new field for each one with a length corresponging to the realtive energy and 
for ii, rel_energ in enumerate(relative_energy_list):
    Q_norm = Q/np.linalg.norm(Q)
    inplane_normal = k_0_ref - np.dot(k_0_ref, Q)/np.linalg.norm(Q)**2*Q
    inplane_normal = inplane_normal / np.linalg.norm(inplane_normal)
    theta_ii = np.arcsin(lmbd/(rel_energ+1)/4/np.pi*np.linalg.norm(Q))
    k_0 = (np.cos(theta_ii)* inplane_normal - np.sin(theta_ii)* Q_norm)*2*np.pi/lmbd*(rel_energ+1)

    params[f'Mode {ii:d}'] = {'number':ii, 'k_0':k_0, 'lmbd':lmbd/(1+rel_energ), 'weigth':abs_weight[ii]}

####### Save it to human (and machine) readable file ##########
par.par_write(params)


## Define incident beam

I'm just using my "normal" ID06 beam.

In [ ]:
## Do a quick simulation of a beam by specifying the aperture of an up-stream 1D lens:
from scipy.special import erf
import os
import pickle

print(params['Geometry']['step_sizes'][0])

# Geometry
condenser_distance = 1000
# Aperture
gauss_sig_param = 2/np.sqrt(50)
cutoff_distance = 0.15

# coord array in condenser plane
x_sample = np.arange(params['Geometry']['grid_shape'][0]) * params['Geometry']['step_sizes'][0]
x_cond = np.fft.fftshift(np.fft.fftfreq(params['Geometry']['grid_shape'][0])/ params['Geometry']['step_sizes'][0]) * params['Beam']['lmbd_ref'] * condenser_distance
Apert = np.exp(-(x_cond)**2/2/gauss_sig_param**2)
# # Add some spherical aberration for fun
# A = A * np.exp(1j*(x_cond*6)**4)
Apert[np.abs(x_cond)>cutoff_distance] = 0

E_0_norm = np.fft.fftshift(np.fft.fft(np.fft.ifftshift(Apert)))

y_sample = np.arange(params['Geometry']['grid_shape'][1]) * params['Geometry']['step_sizes'][1]

y_width = 19.5*1e-3
y_mid = np.mean(y_sample)
y_smooth_param = 0.3*1e-3
y_profile = 1-erf((np.abs(y_sample-y_mid)-y_width/2)/y_smooth_param)

E_0_indcident = E_0_norm[:,np.newaxis]*y_profile[np.newaxis, :]


plt.imshow(np.abs(E_0_indcident))

# Save beam


beam_name = 'incident_1'

if not os.path.isdir(root_dir + 'incident_field'):
    os.makedirs(root_dir + 'incident_field')

with open(root_dir + 'incident_field/' + beam_name + '.npy', 'wb+') as fid:
    pickle.dump(E_0_indcident, fid)


## Specify optical parameters

Look up scattering constants for alimunium as a function of energy

In [ ]:
############### CALCULATE MATERIAL OPTICAL PROPERTIES ####################

cif_file= params['I/O']['root_dir'] + '/9012955.cif'
''' get scatter function based on dan's diffraction
    Dan's diffraction reads cif files and is used to calculate correct atomic positions for scattering
    Dan's diffraction does not include a function for anomalous scattering, but this has been made based on xrddb and is assigned separately
'''

import Dans_Diffraction
import dfxm_fwrd_sim.structure_factor



for ii in range(params['Beam']['N_modes']):
    print(params[f'Mode {ii:d}']['lmbd'])
    xtl = dfxm_fwrd_sim.structure_factor.setup_dans_diffraction(cif_file, float(params[f'Mode {ii:d}']['lmbd'])*1000 ) # this one runs in microns still
    chi_h = xtl.Scatter.x_ray_anomalous(params['Geometry']['hkl'] )[0]
    chi_hoverline = xtl.Scatter.x_ray_anomalous(-params['Geometry']['hkl'] )[0]

    print(chi_h)
    prod = np.sqrt(chi_h*chi_hoverline)
    print(prod)

    # see https://journals.iucr.org/q/issues/1949/02/00/a00117/a00117.pdf equation 8 and 9

    params[f'Mode {ii:d}']['chi_h_Re'] = np.real(prod)*np.cos(20*np.pi/180)
    params[f'Mode {ii:d}']['chi_h_Im'] = np.imag(prod)*np.cos(20*np.pi/180)
    params[f'Mode {ii:d}']['chi_0_Re'] = 3.2e-6  ### !!!!!!! LOOK THESE VALUES UP SOMWHERE DONT BE LAZY MADS
    params[f'Mode {ii:d}']['chi_0_Im'] = 9.0e-10
    params[f'Mode {ii:d}']['incident_field'] = root_dir + 'incident_field/' + beam_name + '.npy'

par.par_write(params)

## Define sample


In [ ]:
import dfxm_fwrd_sim.test_samples

R_disloc_sim = U_sim @ disloc_to_latt

Q = params[f'Geometry']['Q']
b = np.dot(A, np.array([0.5, -0.5, 0]) )
bnorm = np.linalg.norm(b)

L = params['Geometry']['step_sizes'][2] * params['Geometry']['grid_shape'][2]
Lx = params['Geometry']['step_sizes'][0] * params['Geometry']['grid_shape'][0]
Ly = params['Geometry']['step_sizes'][1] * params['Geometry']['grid_shape'][1]
mid_x = 0.5 - np.tan(theta)*0.5*L/Lx


mid = [mid_x, 0.5, 0.5]
dist_between_dislocs = 0.004
number_of_disloc = 301//4

displacement_field = dfxm_fwrd_sim.test_samples.EdgeDislocationWall(params, mid, R_disloc_sim, bnorm,dist_between_dislocs, number_of_disloc)
plt.figure(figsize = (10, 10))
plt.imshow(np.abs(displacement_field[:,:,501])**2)

In [ ]:
# Save displacement field
disp_field_name = f'{round(dist_between_dislocs*1e3):d}_distance_{number_of_disloc:d}_number'
with open(root_dir + disp_field_name + '.npy', 'wb') as fid:
    pickle.dump(displacement_field, fid)

# Update ini file
params['I/O']['displacement_field'] = root_dir + disp_field_name + '.npy'
par.par_write(params)


## Imaging system

Define a lens-object and an imaging geometry

In [ ]:
import dfxm_fwrd_sim.CRL_sim as CRL
# Calculate individual lenslet parameters:
R = 0.05 #mm
central_wavelength_mm = params['Beam']['lmbd_ref'] # mm
material = 'Be'

# Pack parameters into a dictionary
params['Optics'] = {'Material':material, 'lmbd':central_wavelength_mm}

# Look up optical constants
CRL.NIST_Reader( params['Optics'], datafile_directory = r'forward_simulation_polychromatic-master\attenuation_datafiles')

# Calculate individual lenslet parameters
f = R / 2 / params['Optics']['delta'] 
sig_a = 1 / 2 * np.sqrt( R*params['Optics']['lmbd']/np.pi/params['Optics']['beta']) 

print(params['Optics'])

# Define a simple lens
lens_description = []
#lens_description.append({'kind':'aberration_function', 'function':aber_1})
lens_description.append({'kind':'lens box', 'N':88, 'f':f, 'T':1.600, 'sig_a':sig_a})
#lens_description.append({'kind':'aberration_function', 'function':aber_2})
lens_description.append({'kind':'square aperture', 'width':0.1}) 


# Find focus condition
print(f'2theta = {2*theta*180/np.pi} degrees')
tot_dist = 4778/np.cos(2*theta)
print(f'optical path = {tot_dist} mm')
a = 200;b= 250

# Focus plot to test ranges
d1_list = np.linspace(a,b,100)
d2_list = [CRL.find_focal_distance(d1, lens_description)[0] for d1 in d1_list]
plt.plot(d1_list, d1_list+d2_list)
plt.plot([d1_list[0], d1_list[-1]], [tot_dist]*2)
plt.show()

# Calculate focus position
d1_fl = CRL.determine_focus_condition(tot_dist, lens_description, a, b)
d2_fl, a_fl = CRL.find_focal_distance(d1_fl, lens_description)
M = d2_fl/d1_fl/a_fl
params['Optics']['M'] = M
d1_eff = tot_dist/(np.abs(M)+1)

# Shift origin in lens description to eff. thin lens pos.
lens_description_eff = list(lens_description)
lens_description_eff.insert(0, {'kind':'free space', 'length':(d1_fl - d1_eff)})
params['Optics']['d1'] = d1_eff

print(f'M = {M}')

CRL.draw_lens(d1_fl, lens_description_eff)


# Save objective lens
obj_lens_name = 'perf_CRL'
with open(root_dir + obj_lens_name + '.npy', 'wb+') as fid:
    pickle.dump(lens_description_eff, fid)

# Update ini file
params['I/O']['obj_lens'] = root_dir + obj_lens_name + '.npy'
par.par_write(params)
print(M)
print(d1_fl)

## Run integrations

In [ ]:
# import dfxm_fwrd_sim.parameter_parser as par
# import dfxm_fwrd_sim.integrate as integrate

# par_fn = '/u/data/madsac/Foward_simulation_polychromatic/al_4.0_mu/al_1.ini'
# params = par.par_read(par_fn)
# params['Status']['processes'] = 1
# par.par_write(params) 

# for phi in [0, 25, 50, 75, 100, 125, 150, 175, 200, 250, 275, 300]:
#     phi = phi*1e-6
#     for n in range(11):
        
#         integrate.integrate_mode_n(n, par_fn, phi)


# import dfxm_fwrd_sim.parameter_parser as par
import dfxm_fwrd_sim.integrate as integrate
from tqdm import tqdm
# Maybe comment these two lines?
# par_fn = 'forward_simulation_polychromatic-master/al_1.0_mu/al_1.ini'
# params = par.par_read(par_fn)
# params['Status']['processes'] = 11
# par.par_write(params)
""" This is heavy computing (76 hours)
for phi in tqdm([0, 25, 50, 75, 100, 125, 150, 175, 200, 250, 275, 300], desc = 'Running'):
    phi = phi*1e-6
    integrate.integrate_parallel(root_dir + par_fn, phi)
"""
# params

for i in tqdm(range(params['Beam']['N_modes'])):
    integrate.integrate_mode_n(i, root_dir + par_fn, 300*1e-6)

integrate.integrate_parallel(root_dir + par_fn, 300*1e-6)


In [ ]:
# Plot result:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import dfxm_fwrd_sim.parameter_parser as par
# par_fn = 'forward_simulation_polychromatic-master/al_1.0_mu/try_al_2.ini'
# params = par.par_read(par_fn)
intens = np.zeros(params['Geometry']['grid_shape'][:2]) 

mu = 300

for modenumber in range(11):
    with open(params['I/O']['root_dir'] + f'integrated_field/phi_{mu:04d}_murad/mode_{modenumber:04d}.npy', 'rb') as fid:
        intens += np.abs(pickle.load(fid))**2 * params[f'Mode {modenumber}']['weigth']


x = np.arange(params['Geometry']['grid_shape'][0])*params['Geometry']['step_sizes'][0]
y = np.arange(params['Geometry']['grid_shape'][1])*params['Geometry']['step_sizes'][1]
fig = plt.figure(figsize = (15, 15))


ax = fig.add_subplot(1, 1, 1)

#plt.imshow(intens,extent = (y[0]*1e3, y[-1]*1e3,x[0]*1e3,x[-1]*1e3))
theta= 20/57
plt.imshow(intens,extent = (y[0]*1e3, y[-1]*1e3,x[-1]*1e3/2/np.sin(theta),x[0]*1e3/2/np.sin(theta)), vmax = 400 )

#b = np.array([1, -1, 0])/np.sqrt(2)
#line_trace = np.cross(ref_to_lab @ b, [0,0,1])

#print(line_trace)
#y = np.array([y[0]*1e3, y[-1]*1e3])
#x = y*line_trace[0]/line_trace[1]
#plt.plot(y, x + 62,'r')

ax.set_aspect('equal')
plt.ylabel('x_lab (µm)')
plt.xlabel('y_lab (µm)')
plt.title(f'φ = {mu} µm')
ax.set_facecolor([1,1,1])

# plt.xlim(8, 12)
# plt.ylim(28, 32)
# plt.savefig(params['I/O']['root_dir'] + '/phi_0.pdf')
plt.colorbar()
plt.savefig("trial1.png")
# plt.show()


In [16]:
import dfxm_fwrd_sim.propagate as prop

# Load parameters
# par_fn = '/u/data/madsac/Foward_simulation_polychromatic/al_2_micron_spacing/al_1.ini'
# params = par.par_read(par_fn)
# par.par_write(params) 
params['Optics']['FOV_cen'] = [params['Geometry']['grid_shape'][ii] * params['Geometry']['step_sizes'][ii]*0.5 for ii in (0,1) ]


# Choose rocking angle
phi = 300e-6
integration_name = f'phi_{phi*1e6:04.0f}_murad'
propagation_name = 'lens_pos_1'

# Rotate initial condition rel. to lens to simulate misalgninment
rot_x = 0.0e-3
rot_y = 0.0e-3

# Call the propagation script

for n in range(params['Beam']['N_modes']):
    prop.propagate(n, params, (rot_x, rot_y), integration_name, propagation_name)


{'kind': 'free space', 'length': -74.43408439694625}
{'kind': 'lens box', 'N': 88, 'f': 21195.203507732826, 'T': 1.6, 'sig_a': 1.1026012773017384}


 34%|███▍      | 30/88 [00:04<00:08,  7.12it/s]


KeyboardInterrupt: 

In [ ]:
# Plot result:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import dfxm_fwrd_sim.parameter_parser as par
# par_fn = '/u/data/madsac/Foward_simulation_polychromatic/al_1.0_mu/al_1.ini'
# params = par.par_read(par_fn)

intens = np.zeros(params['Geometry']['grid_shape'][:2]) 

for modenumber in range(params['Beam']['N_modes']):
    with open(params['I/O']['root_dir'] + f'/integrated_field/phi_0300_murad/mode_{modenumber:04d}.npy', 'rb') as fid:
        intens += np.abs(pickle.load(fid))**2 * params[f'Mode {modenumber}']['weigth']


x = np.arange(params['Geometry']['grid_shape'][0])*params['Geometry']['step_sizes'][0]
y = np.arange(params['Geometry']['grid_shape'][1])*params['Geometry']['step_sizes'][1]
fig = plt.figure(figsize = (15, 15))


ax = fig.add_subplot(1, 1, 1)

#plt.imshow(intens,extent = (y[0]*1e3, y[-1]*1e3,x[0]*1e3,x[-1]*1e3))

plt.imshow(intens,extent = (y[0]*1e3, y[-1]*1e3,x[-1]*1e3/2/np.sin(theta),x[0]*1e3/2/np.sin(theta)))

b = np.array([1, -1, 0])/np.sqrt(2)
line_trace = np.cross(ref_to_lab @ b, [0,0,1])

print(line_trace)
y = np.array([y[0]*1e3, y[-1]*1e3])
x = y*line_trace[0]/line_trace[1]
plt.plot(y, x + 57,'r')

ax.set_aspect('equal')
plt.ylabel('x_lab (µm)')
plt.xlabel('y_lab (µm)')
plt.title('φ = 0 µm')
ax.set_facecolor([1,1,1])
ax.set_ylim([60,80])
# plt.savefig(params['I/O']['root_dir'] + '/phi_0.pdf')
plt.show()
